In [1]:
import pandas as pd
import glob

allfiles = glob.glob('Kickstarter_data/*.csv')
df = pd.concat((pd.read_csv(f) for f in allfiles), ignore_index=True)
print(df.shape)
print(df.keys())
print(df.state.value_counts())

(195614, 37)
Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_starrable',
       'launched_at', 'name', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'location',
       'friends', 'is_backing', 'is_starred', 'permissions'],
      dtype='object')
successful    105680
failed         73634
canceled        8823
live            6876
suspended        601
Name: state, dtype: int64


In [3]:
needed_keys = ['backers_count', 'blurb', 'category', 'converted_pledged_amount', 
               'country', 'deadline', 'goal', 'spotlight', 'staff_pick', 'state', 
               'usd_pledged']
# Filter unnecessary keys
df2 = df[needed_keys]
# Separate finished and live projects
finished = df2.loc[df2.state != 'live']
unfinished = df2.loc[df2.state == 'live']
print(finished.shape)
print(unfinished.shape)

(188738, 11)
(6876, 11)
